ANIME_REC_MODEL_V1

In [ ]:
import pandas as pd
import numpy as np

# nlp
from transformers import Trainer, TrainingArguments

from scipy.spatial.distance import pdist, squareform

from sklearn.preprocessing import MultiLabelBinarizer, OrdinalEncoder

from util.performance import print_mem_mb


In [151]:
og_data = pd.read_csv("../data/anime_dataset_small_nomic.csv")

ord_encoder = OrdinalEncoder()
mlb = MultiLabelBinarizer(sparse_output=True)

Clean the data

In [152]:
# drop image links
data=og_data.drop(list(og_data.filter(regex="Path|Title")),axis=1)

drop_cols = ["Resources", "Char Tags", "Synopsis","Animation Work", "Chief Animation Direction", 
             "Direction", "Chief Direction", "Original Work", "Character Design", "Work",
             "Series Composition", "Animation Character Design", "Original Plan",
             "Music", "synopsis_length",
             "Cast", "Air Date", "Type", "Season", "Year"]

data= data.drop(drop_cols, axis=1)

# convert to lowercase and list
data["Tags"] = data["Tags"].str.lower()
data["Tags"] = data["Tags"].str.split("|")


### Encoding
Encoding the categorical columns first with one hot encoding.

In [153]:
categorical_cols = ["filter_type"]


# encode categorical columns
filter_type = pd.DataFrame(ord_encoder.fit_transform(data[categorical_cols]),
                columns=ord_encoder.get_feature_names_out(categorical_cols))

# drop categorical columns
data = data.drop(categorical_cols, axis=1)

data = pd.concat([data, filter_type], axis=1)


Then, encoding the anime's tags with a MultiLabelBinarizer. There are 1244 tags.

In [154]:
# fuck no
#data["Char Tags"] = data["Char Tags"].str.split("|")

# encode tags
tags = pd.DataFrame.sparse.from_spmatrix(
        mlb.fit_transform(data.pop("Tags")),
                          index=data.index,
                          columns=mlb.classes_)


# removes outer spaces
def strip(col):
  return col.strip()

tags = tags.rename(mapper=strip,axis=1)
data = data.join(tags)


In [155]:
# additional attibutes for distance calculation
attribute_cols = ["Max Rating",	"filter_year","filter_type"]

#test = data[attribute_cols].copy()
#data = data.drop(attribute_cols,axis=1)
#test


Correlation matrix testing

In [156]:
corr = data.corr()

#corr.iloc[0:5][0:5]
#corr["pantyjob"].dropna().sort_values().tail(20)
#corr

In [157]:
anime_name = "Hunter x Hunter"
find_by = "Official Title (en)"
anime = og_data.index[og_data[find_by].str.contains(anime_name)].tolist()

Then calculate euclidean distance to find the closest animes by their distances

In [158]:

# vector of euclidean distances
result = pdist(data, 'euclidean')

# turn euclidean distances into a matrix 
dist_data = pd.DataFrame(squareform(result), index=og_data["Main Title"],columns=og_data["Main Title"])

num_results = 15

dist_data.iloc[anime[0],:].sort_values().head(num_results).tail(num_results-1)
#dist_data
#og_data


Main Title
Hunter x Hunter: Original Video Animation     5.572513
Grappler Baki (2001)                          6.147268
Shadow Skill (1998)                           6.292225
Gekijouban Marco: Haha o Tazunete Sanzenri    6.325512
Captain Tsubasa (2001)                        6.404069
One Piece (2000)                              6.436031
Groove Adventure Rave                         6.492465
Hunter x Hunter: Greed Island                 6.509386
Street Fighter Zero The Animation             6.535143
Gensou Maden Saiyuuki                         6.535756
Tales of Eternia The Animation                6.556897
Shin Chou Kyou Ryo: Condor Hero               6.570114
Himiko-Den                                    6.597158
Kaze no Youjinbou                             6.603666
Name: Hunter x Hunter (1999), dtype: float64

In [159]:
# Testing for mem usage
#print_mem_mb(data)